In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from pathlib import Path

from utils import PositionalEncoder, load_data, split_data
from models import TrainConfig, RNNClassifier, evaluate_rnn_model, save_model, load_model, plot_results

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print("Device: cuda")
        print(torch.cuda.get_device_name(i))
else:
    print("Device: cpu")

Device: cuda
NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
# Use this if running on Colab
# from google.colab import drive
# drive.mount('/content/drive')
# base_dir = "/content/drive/MyDrive/data/power"

# Use this if running locally
base_dir = "data/train/power"

In [3]:
# Load packages
file_list = [
    'power-gb-train.tsv',
    'power-ua-train.tsv',
    # 'power-fr-train.tsv',
    # 'power-nl-train.tsv',
]

full_data = load_data(folder_path=base_dir, file_list=file_list,text_head='text_en')
train_dev_raw, test_raw = split_data(full_data, test_size=0.2, random_state=0)
train_raw, dev_raw = split_data(train_dev_raw, test_size=0.2, random_state=0)


Load power-gb-train.tsv...
Load power-ua-train.tsv...


In [4]:

print("Prepare data encoder...")
train_encoder = PositionalEncoder()
train_encoder.fit(train_raw.texts)

Prepare data encoder...


PositionalEncoder(tokenizer=<function PositionalEncoder.build_tokenizer.<locals>.simple_tokenizer at 0x7f199b576340>,
                  vocabulary={'': 1, '!': 63796, '"': 68750, '#': 35922,
                              '$': 33148, '%': 10679, '&': 21783, "'": 72787,
                              '(': 19166, ')': 56035, '*': 7770, '+': 52968,
                              ',': 14681, '-': 50292, '.': 11993, '/': 6372,
                              '0': 15562, '00': 62551, '000': 3284,
                              '0000': 50639, '000th': 32236, '001': 995,
                              '002': 8219, '0023': 705, '0025': 9448,
                              '0026': 42244, '0029': 61776, '004': 2828,
                              '0040': 34655, '005': 66925, ...})

In [8]:
train_dataloader = DataLoader(train_raw, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_raw, batch_size=128, shuffle=True)

# Prepare baseline config
train_config = TrainConfig(
    optimizer_params = {'lr': 0.01},
    num_epochs       = 10,
    early_stop       = False,
    violation_limit  = 5
)

# Train baseline model
model_lstm = RNNClassifier(
    rnn_network         = nn.LSTM,
    word_embedding_dim  = 32,
    hidden_dim          = 64,
    bidirectional       = False,
    dropout             = 0,
    encoder             = train_encoder,
    device              = 'cuda'
)

if Path('models/model_lstm.pt').exists():
    model_lstm = load_model(model_lstm, 'model_lstm')
else:
    model_lstm.fit(train_dataloader, train_config, no_progress_bar=False)
    save_model(model_lstm, "model_lstm")


model_lstm_result = evaluate_rnn_model(model_lstm, test_dataloader, train_encoder)
print(model_lstm_result)

np.save('models/model_lstm_results.npy', model_lstm_result)



Epoch 1:   0%|          | 0/222 [00:00<?, ?batch/s]/media/hapham/Work/study/power-identification/utils.py:94: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  tokens_sparse = torch.sparse_csr_tensor(crow, col, token_val, size=mat_size, dtype=torch.long)
Epoch 5: 100%|██████████| 222/222 [01:40<00:00,  2.21batch/s, batch_accuracy=0.878, loss=45.6]


In [12]:
plot_results(model_lstm, train_config, train_dataloader)

alt.HConcatChart(...)

In [13]:

# Prepare baseline config
gru_train_config = TrainConfig(
    optimizer_params = {'lr': 0.01},
    num_epochs       = 10,
    early_stop       = False,
    violation_limit  = 5
)

# Train baseline model
model_gru = RNNClassifier(
    rnn_network         = nn.GRU,
    word_embedding_dim  = 32,
    hidden_dim          = 64,
    bidirectional       = False,
    dropout             = 0,
    encoder             = train_encoder,
    device              = 'cuda'
)

if Path('models/model_gru.pt').exists():
    model_gru = load_model(model_gru, 'model_gru')
else:
    model_gru.fit(train_dataloader, gru_train_config, no_progress_bar=False)
    save_model(model_gru, "model_gru")


model_gru_result = evaluate_rnn_model(model_gru, test_dataloader, train_encoder)
print(model_gru_result)

np.save('models/model_gru_results.npy', model_gru_result)



100%|██████████| 71/71 [00:16<00:00,  4.43batch/s]

(0.6836327314376831, 0.9433239102363586, 0.7662816643714905)


In [25]:
plot_results(model_gru, gru_train_config, train_dataloader)

alt.HConcatChart(...)